<a href="https://colab.research.google.com/github/AntarikshVerma/AI_Practice_Repo/blob/main/History_Teacher_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Azure OpenAI — History Teacher Persona
This notebook shows how to call **Azure OpenAI** and create a history teaching assistant using it.



In [ ]:
!pip install --quiet openai

In [ ]:
import os
from google.colab import userdata
open_ai_key=userdata.get('OPENAI_API_KEY_GPT_4')
open_ai_base=userdata.get('OPENAI_API_URL_GPT_4')

api_key = open_ai_key
endpoint = open_ai_base
deployment = "gpt-4o"


In [ ]:

from openai import AzureOpenAI
client = AzureOpenAI(api_key=api_key, azure_endpoint=endpoint, api_version="2024-05-01-preview")


#### Default system prompt for History Teacher persona

In [ ]:
default_system_prompt = """You are a knowledgeable, patient, and engaging **History Teacher**.
- Always adopt a clear, respectful, educational tone.
- Answer only history-related questions (events, people, causes, timelines, context, significance, methods of historical analysis).
- If the user asks about non-history topics (medical advice, legal, real-time political campaigning, coding how-tos, etc.), politely refuse and redirect to historical context where possible.
- Refuse or do not provide content that is unsafe or requests facilitation of wrongdoing.
- When relevant, cite widely-known dates, timelines, and provide concise summaries first and then expanded explanation sections.
- Use simple language for learners, but be ready to provide advanced detail when asked.
"""


##### Azure Open AI Invocation

In [ ]:
def _call_chat_api(messages, max_tokens=400, temperature=0.2):
    resp = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    # Most modern clients return choices[0].message.content
    return resp.choices[0].message.content


In [ ]:
def ask_history_teacher(question: str, system_prompt: str = None, max_tokens: int = 400, temperature: float = 0.2) -> str:
    sys_prompt = system_prompt or default_system_prompt
    messages = [
        {"role":"system", "content": sys_prompt},
        {"role":"user", "content": question},
    ]
    try:
        raw = _call_chat_api(messages, max_tokens=max_tokens, temperature=temperature)
        # safety: ensure result is string
        return raw.strip() if isinstance(raw, str) else str(raw)
    except Exception as exc:
        return f"[ERROR] API call failed: {exc}"


##### Usage

In [ ]:
# Example 1: plain history question
print("Example 1: History question")
print(ask_history_teacher("Who was the first emperor of the Roman Empire?"))
print("\n" + "="*60 + "\n")



In [ ]:
# Example 2: non-history question to test guardrails
print("Example 2: Non-history question (should be refused/redirected)")
print(ask_history_teacher("What is 2+2?"))


In [ ]:

# Example 3: multi-part question
print("\n" + "="*60 + "\n")
print("Example 3: Multi-part history question")
print(ask_history_teacher("Explain the main causes of World War I in simple terms, then suggest two primary sources a student could read."))
